In [1]:
import filepath_

In [2]:
import torch
import numpy as np
import sympy as sp
from abc import ABC
from typing import List, Dict, Tuple
from panda3d.core import VBase3
from panda3d.core import Vec3
from panda3d.core import Quat
from panda3d.core import invert

from panda3d_game.game_object import GameObject, PhysicsGameObject
from panda3d.core import TransformState
# 
from util.geometry import batch_transform, createPosIndicatorNPth
from art.basic import geom_frm_faces, create_cylinder_node, create_cylinder
from panda3d.core import GeomPrimitive
from panda3d.core import (
    Geom,
    GeomNode,
    GeomTriangles,
    GeomVertexData,
    GeomVertexFormat,
    GeomVertexWriter,
    GeomEnums, Vec3, LPoint3f
)    
from panda3d.core import GeomNode, NodePath, GeomVertexReader, BitMask32
from art.basic import uv_curve_surface, uv_curve_surface_lambda
from panda3d.bullet import (
    BulletWorld, BulletRigidBodyNode, BulletTriangleMesh, 
    BulletTriangleMeshShape, BulletDebugNode,
      BulletHingeConstraint, BulletGenericConstraint,
    BulletShape, BulletMinkowskiSumShape,BulletCylinderShape,
BulletConvexHullShape,BulletTriangleMeshShape, BulletTriangleMesh
)
from panda3d_game.constraints import FixedConstraint
from panda3d_game.app import UniversalGravitySpace
from qpanda3d import QControl
from ui.qtui import *
from panda3d.bullet import BulletRigidBodyNode
from panda3d.bullet import BulletSphereShape, BulletCylinderShape
from console.physics_room_console import PhyscRoomConsole

In [3]:
# from icosahedron import *
from art.procedural_art.fractal_landscape.panda3d_spherical import *
from panda3d.core import DirectionalLight, Vec3, PointLight
from sympy.physics.units import (
    kilometer, meter,centimeter,
    gram, kilogram, tonne,
    newton, second
)

In [4]:
mesh = SphereMeshObject(R=100)

In [5]:
mesh.uniformSplit(5)

iter 0
iter 1
iter 2
iter 3
iter 4


In [6]:
# mesh.setBulletShape()
# mesh.setGeom()
mesh.setBulletAndGeom()

In [7]:

        

def addHyperEdgeAsTriangle(hp:HyperEdge):
    rho = 1
    x = hp.x
    p = x.theta
    y = hp.y
    z = hp.z
    

class ShipScene(
    # StarScene, 
    UniversalGravitySpace, 
    QControl
    # ControlShowBase
):
    def __init__(self, num_iterations=20):
        # StarScene.__init__(self)
        unit = {
            "mass" : tonne,
            "length" : 10*meter,
            "time": 1 * second,
            # "force" : sp.Number(1e3) * newton
        }
        G_game = 0.001
        UniversalGravitySpace.__init__(self, unit, G_game, 20, 1/(60*20))
        QControl.__init__(self)
       
       
        point_light = PointLight('light')
        
        point_light.setColor((1, 1, 1, 1))  # 设置光源颜色 (白色)
        point_light.setShadowCaster(True)
        light_np = self.render.attachNewNode(point_light)
        light_np.setPos(2, 2, 2)
        self.render.setLight(light_np)  # 将光源应用到场景
        
        # point_light.set_sc
        dire_light =DirectionalLight('light')
        dire_light.setDirection((-1,-1,-1))
        dire_light.set_color((1, 1, 1, 1.0)) 
        dire_light.setShadowCaster(True)
        light_dr = render.attachNewNode(dire_light)
        # light_dr.setPos(1, 2, 6)  # 设置光源位置
        
        self.render.setShaderAuto()
        self.render.setLight(light_dr)
        self.startQt()
        self.m = mesh
        self.m.rigid_np.reparentTo(self.rdr_scene)
        self.bullet_world.attachRigidBody(mesh.rigid_node)

        # self.ship.toBulletWorld(self.bullet_world)
        # self.shell.toBulletWorld(self.bullet_world)
        # self.planet1.toBulletWorld(self.bullet_world)
        # self.taskMgr.add(self.update_torque_)
       

        debug_node = BulletDebugNode('Debug')
        debug_node.showWireframe(True)
        debug_node.showConstraints(True)
        debug_node.showBoundingBoxes(False)
        debug_node.showNormals(False)
        debug_np = self.rdr_scene.attach_new_node(debug_node)
        self.bullet_world.set_debug_node(debug_node)
        debug_np.show()
class ShipView(RawQtGUI):
    def get_game(self):
        return ShipScene()

    def get_console(self):
        return PhyscRoomConsole(showbase=self.panda3d) 

In [ ]:
if __name__ == '__main__':
    # torch.set_printoptions(precision=16, sci_mode=False)
    import sys
    app = QApplication(sys.argv)
    window = ShipView()
    window.show()
    sys.exit(app.exec_())

Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager


In [6]:
v = mesh.getVdata()

coord (12, 3)
arr (12, 4)


In [7]:
v

12 rows: [ vertex(3f) color(4b) ]

In [8]:
varr = v.modifyArray(0)

In [9]:
vhandle = varr.modifyHandle()

In [18]:
vbyte = vhandle.getData()
varr_np = np.frombuffer(vbyte, np.float32)
varr_np.resize(12,7)
varr_np[:,:3]

array([[ 0.0000000e+00,  0.0000000e+00,  1.0000000e+02],
       [ 8.9442726e+01,  0.0000000e+00,  4.4721352e+01],
       [ 2.7639317e+01,  8.5065086e+01,  4.4721352e+01],
       [-7.2360687e+01,  5.2573109e+01,  4.4721352e+01],
       [-7.2360680e+01, -5.2573128e+01,  4.4721352e+01],
       [ 2.7639332e+01, -8.5065086e+01,  4.4721352e+01],
       [ 7.2360680e+01,  5.2573116e+01, -4.4721352e+01],
       [-2.7639326e+01,  8.5065086e+01, -4.4721352e+01],
       [-8.9442726e+01, -7.8193307e-06, -4.4721352e+01],
       [-2.7639332e+01, -8.5065086e+01, -4.4721352e+01],
       [ 7.2360680e+01, -5.2573120e+01, -4.4721352e+01],
       [-8.7422777e-06, -0.0000000e+00, -1.0000000e+02]], dtype=float32)

In [12]:
varr_np.shape

(84,)

In [13]:
from panda3d.core import GeomNode, GeomVertexReader
vread = []
vertex_reader = GeomVertexReader(v, "vertex")
while not vertex_reader.is_at_end():
    x, y, z = vertex_reader.get_data3f()  # Read the vertex position as an (x, y, z) tuple
    vread.append((x, y, z))

In [19]:
from panda3d.core import GeomNode, GeomVertexReader
vreadc = []
vertex_reader = GeomVertexReader(v, "color")
while not vertex_reader.is_at_end():
    x, y, z = vertex_reader.get_data3f()  # Read the vertex position as an (x, y, z) tuple
    vreadc.append((x, y, z))

In [14]:
len(vread)

21

In [15]:
vread

[(0.0, 0.0, 100.0),
 (0.0, 0.0, 0.0),
 (0.0, 44.721351623535156, 0.0),
 (0.0, 0.0, 27.63931655883789),
 (44.721351623535156, 0.0, 0.0),
 (0.0, -72.36068725585938, 52.5731086730957),
 (0.0, 0.0, 0.0),
 (-72.36067962646484, -52.57312774658203, 44.721351623535156),
 (0.0, 0.0, 0.0),
 (-85.0650863647461, 44.721351623535156, 0.0),
 (0.0, 0.0, 72.36067962646484),
 (-44.721351623535156, 0.0, 0.0),
 (0.0, -27.639326095581055, 85.0650863647461),
 (0.0, 0.0, 0.0),
 (-89.4427261352539, -7.819330676284153e-06, -44.721351623535156),
 (0.0, 0.0, 0.0),
 (-85.0650863647461, -44.721351623535156, 0.0),
 (0.0, 0.0, 72.36067962646484),
 (-44.721351623535156, 0.0, 0.0),
 (0.0, -8.742277714191005e-06, -0.0),
 (0.0, 0.0, 0.0)]

In [20]:
vreadc

[(0.0, 0.0, 0.0),
 (0.6784313917160034, 0.8862745761871338, 0.6980392336845398),
 (0.0, 0.0, 0.0),
 (0.32549020648002625, 0.12941177189350128, 0.6666666865348816),
 (0.0, 0.0, 0.0),
 (0.6666666865348816, 0.8862745761871338, 0.19607844948768616),
 (0.0, 0.0, 0.0),
 (0.0, 0.0, 0.0),
 (0.3529411852359772, 0.11372549831867218, 0.8666667342185974),
 (0.0, 0.0, 0.0),
 (0.874509871006012, 0.29019609093666077, 0.32156863808631897),
 (0.0, 0.0, 0.0),
 (0.6666666865348816, 0.8862745761871338, 0.19607844948768616),
 (0.0, 0.0, 0.0),
 (0.0, 0.0, 0.0),
 (0.3529411852359772, 0.11372549831867218, 0.8666667342185974),
 (0.0, 0.0, 0.0),
 (0.8784314393997192, 0.29019609093666077, 0.32156863808631897),
 (0.0, 0.0, 0.0),
 (0.0, 0.0, 0.7843137979507446),
 (0.0, 0.0, 0.0)]

In [8]:
mesh.toGeom()

Geom [ GeomTriangles ], 5 faces

In [10]:
"""
:gobj(error): GeomTriangles references vertices up to 64982, but GeomVertexData has only 48738 rows!
Assertion failed: primitive->check_valid(cdata->_data.get_read_pointer(current_thread)) at line 364 of panda/src/gobj/geom.cxx
"""

'\n:gobj(error): GeomTriangles references vertices up to 64982, but GeomVertexData has only 48738 rows!\nAssertion failed: primitive->check_valid(cdata->_data.get_read_pointer(current_thread)) at line 364 of panda/src/gobj/geom.cxx\n'

In [11]:
len(mesh.verts)

12

In [12]:
x = sp.Symbol('x')
e = (64984*x)/ (48738*x)
e.simplify()
# seems that it is format issue

4/3

In [13]:
mesh.getXYZ().shape

torch.Size([12, 3])

In [14]:
tp = mesh.getThetaPhi()


In [15]:
xyz = sphr2cart_pt(tp[:,0], tp[:,1])
xyz * mesh.getHeight()

RuntimeError: The size of tensor a (3) must match the size of tensor b (12) at non-singleton dimension 1

In [ ]:
xyz.shape